In [1]:
import dags

In [2]:
def utility(consumption, leisure, leisure_prefs):
    return consumption + leisure_prefs["weight"] * leisure ** leisure_prefs["exponent"]


def leisure_prefs(params):
    return params["preferences"]["leisure"]


def leisure(params):
    return params["time_budget"] - params["work"]["hours"]


def income(params):
    return params["work"]["hourly_wage"] * params["work"]["hours"]


def consumption(income):
    return income


def unrelated(working_hours):
    raise NotImplementedError()


In [3]:
from estimagic import maximize
from copy import deepcopy

In [26]:
start_params = {
    "preferences": {
        "leisure": {
            "weight": 2,
            "exponent": 0.5
        }
    },
    "work": {
        "hourly_wage": 25,
        "hours": 2_000
    },
    "time_budget": 24 * 7 * 365,
}


In [27]:
model = dags.concatenate_functions(
    functions=[utility, unrelated, leisure, leisure_prefs, consumption, income],
    targets=["utility", "consumption"],
    return_type="dict"
)


In [28]:
model(params=start_params)

{'utility': 50487.11394970787, 'consumption': 50000}

In [29]:
u = dags.concatenate_functions(
    functions=[utility, leisure, leisure_prefs, income, consumption],
    targets="utility"
)

In [30]:
def return_all_but_working_hours(params):
    out = deepcopy(params)
    del out["work"]["hours"]
    return out

In [31]:
result = maximize(
    criterion=u,
    params=start_params,
    constraints=[
        {
            "selector": return_all_but_working_hours, 
            "type": "fixed"
        },
        {
            "selector": lambda p: [p["work"]["hours"], p["time_budget"]],
            "type": "decreasing"
        },
    ],
    lower_bounds={"work": {"hours": 0}},
    algorithm="nlopt_neldermead",
    logging="log.db",
    log_options={"if_database_exists": "replace"}
)

In [32]:
result

Maximize with 1 free parameters terminated successfully after 12 criterion evaluations.

The value of criterion improved from 50487.11394970787 to 1533000.0.

The nlopt_neldermead algorithm reported: Optimizer stopped because convergence_relative_params_tolerance or convergence_absolute_params_tolerance was reached

Independent of the convergence criteria used by nlopt_neldermead, the strength of convergence can be assessed by the following criteria:

                          one_step   five_steps 
relative_criterion_change   0***        0.698   
relative_params_change      0***       0.6983   
absolute_criterion_change   0***     1.07e+06   
absolute_params_change      0***    4.282e+04   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)

In [33]:
result.params

{'preferences': {'leisure': {'weight': 2.0, 'exponent': 0.5}},
 'work': {'hourly_wage': 25.0, 'hours': 61320.0},
 'time_budget': 61320.0}

In [34]:
model(params=result.params)

{'utility': 1533000.0, 'consumption': 1533000.0}